In [7]:
using MXNet
data = mx.Variable(:data)
fc1  = mx.FullyConnected(data, name=:fc1, num_hidden=128)
act1 = mx.Activation(fc1, name=:relu1, act_type=:relu)
fc2  = mx.FullyConnected(act1, name=:fc2, num_hidden=64)
act2 = mx.Activation(fc2, name=:relu2, act_type=:relu)
fc3  = mx.FullyConnected(act2, name=:fc3, num_hidden=10)

SymbolicNode fc3

In [9]:
mlp  = mx.SoftmaxOutput(fc3, name=:softmax)
mlp = @mx.chain mx.Variable(:data)             =>
  mx.FullyConnected(name=:fc1, num_hidden=128) =>
  mx.Activation(name=:relu1, act_type=:relu)   =>
  mx.FullyConnected(name=:fc2, num_hidden=64)  =>
  mx.Activation(name=:relu2, act_type=:relu)   =>
  mx.FullyConnected(name=:fc3, num_hidden=10)  =>
  mx.SoftmaxOutput(name=:softmax)
batch_size = 100
include("/home/dreuter/.julia/packages/MXNet/examples/mnist/mnist-data.jl")
train_provider, eval_provider = get_mnist_providers(batch_size)

(MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Nothing} @0x0000000003b644c0), Tuple{Symbol,Tuple}[(:data, (784, 100))], Tuple{Symbol,Tuple}[(:softmax_label, (100,))], 100, true, true), MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Nothing} @0x0000000004184250), Tuple{Symbol,Tuple}[(:data, (784, 100))], Tuple{Symbol,Tuple}[(:softmax_label, (100,))], 100, true, true))

In [10]:
model = mx.FeedForward(mlp, context=mx.cpu())
optimizer = mx.SGD(η=0.1, μ=0.9, λ=0.00001)
mx.fit(model, optimizer, train_provider, n_epoch=20, eval_data=eval_provider)

┌ Info: Start training on Context[CPU0]
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:367
┌ Info: Initializing parameters...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:374
┌ Info: Creating KVStore...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:380
┌ Info: TempSpace: Total 0 MB allocated on CPU0
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:416
┌ Info: Start training...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:472
┌ Info: == Epoch 001/020 ==========
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:546
┌ Info: ## Training summary
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:548
┌ Info:           accuracy = 0.7568
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:550
┌ Info:               time = 2.3583 seconds
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:552
┌ Info: ## Validation summary
└

### Conv-net

In [11]:
# input
data = mx.Variable(:data)

# first conv
conv1 = @mx.chain mx.Convolution(data, kernel=(5,5), num_filter=20)  =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# second conv
conv2 = @mx.chain mx.Convolution(conv1, kernel=(5,5), num_filter=50) =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))


SymbolicNode pooling1

In [12]:
# first fully-connected
fc1   = @mx.chain mx.Flatten(conv2) =>
                  mx.FullyConnected(num_hidden=500) =>
                  mx.Activation(act_type=:tanh)

# second fully-connected
fc2   = mx.FullyConnected(fc1, num_hidden=10)

# softmax loss
lenet = mx.Softmax(fc2, name=:softmax)


SymbolicNode softmax

In [13]:
batch_size = 100
include("/home/dreuter/.julia/packages/MXNet/examples/mnist/mnist-data.jl")
train_provider, eval_provider = get_mnist_providers(batch_size; flat=false)


(MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Nothing} @0x00000000057de080), Tuple{Symbol,Tuple}[(:data, (28, 28, 1, 100))], Tuple{Symbol,Tuple}[(:softmax_label, (100,))], 100, true, true), MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Nothing} @0x00000000044701f0), Tuple{Symbol,Tuple}[(:data, (28, 28, 1, 100))], Tuple{Symbol,Tuple}[(:softmax_label, (100,))], 100, true, true))

In [14]:
# fit model
model = mx.FeedForward(lenet, context=mx.gpu())

# optimizer
optimizer = mx.SGD(η=0.05, μ=0.9, λ=0.00001)

# fit parameters
mx.fit(model, optimizer, train_provider, n_epoch=20, eval_data=eval_provider)

┌ Info: Start training on Context[GPU0]
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:367
┌ Info: Initializing parameters...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:374
┌ Info: Creating KVStore...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:380
┌ Info: TempSpace: Total 17 MB allocated on GPU0
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:416
┌ Info: Start training...
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:472
[05:01:01] src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:97: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
┌ Info: == Epoch 001/020 ==========
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:546
┌ Info: ## Training summary
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:548
┌ Info:           accuracy

In [15]:
probs = mx.predict(model, eval_provider)

┌ Info: TempSpace: Total 5 MB allocated on GPU0
└ @ MXNet.mx /home/dreuter/.julia/packages/MXNet/XoVCW/src/model.jl:167


10×10000 Array{Float32,2}:
 4.57196e-13  2.36411e-8   1.39104e-9   …  5.84189e-12  5.01485e-9 
 2.64309e-11  1.47528e-9   1.0             1.4269e-15   2.84766e-11
 5.08313e-12  1.0          3.51201e-10     3.07198e-14  8.19767e-10
 1.39607e-8   1.37036e-10  2.5803e-13      6.01223e-9   3.63981e-10
 9.56771e-12  4.40153e-15  1.49125e-7      6.18142e-17  4.36802e-10
 1.59484e-12  2.0136e-15   1.10558e-12  …  0.999959     1.11103e-8 
 9.39286e-20  1.10694e-9   1.56691e-9      6.73486e-8   1.0        
 1.0          1.66438e-15  5.00241e-9      5.80483e-15  7.43109e-15
 1.8015e-13   4.97472e-9   2.69126e-8      4.12388e-5   1.77438e-8 
 1.77884e-8   5.01488e-14  1.41487e-12     5.23575e-14  7.74133e-16